# Comparing Models and Evaluating Metrics

## Importing Libs

In [39]:
# Standard libs
import os
import numpy as np
import pandas as pd

# Model Selection libs
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Feature Engine libs
from feature_engine.encoding import OneHotEncoder
from feature_engine.imputation import CategoricalImputer
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.imputation import ArbitraryNumberImputer, MeanMedianImputer

# Preprocessing libs
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


# Machine Learning libs
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Machine Learning Metrics libs
import sklearn
from sklearn.metrics import get_scorer_names
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import (
    r2_score, mean_absolute_error, mean_squared_error,
    median_absolute_error, mean_absolute_percentage_error,
    max_error, mean_squared_log_error
)

In [2]:
df_train = pd.read_csv('/Users/dellacorte/py-projects/data-science/supervised-learning-regression-reference/databases/house-price.csv', sep=";")
df_train.head()

,zoneamento,tam_terreno,forma_terreno,qualidade_geral,condicao,ano_construcao,qualidade_aquecedor,ar_condicionado,tam_primeiro_andar,tam_segundo_andar,...,qtde_banheiros,qtde_comodos,qtde_lareiras,qtde_carros_garagem,tam_garagem,tam_piscina,qualidade_piscina,mes_venda,ano_venda,preco
0,RL,785.03035,Reg,7,5,2003,Ex,Y,79.524968,79.339162,...,30,8,0,2,50.910844,0.0,NotAv,2,2008,8.465100e+05
1,RL,891.86880,Reg,6,8,1976,Ex,Y,117.243586,0.000000,...,2,6,1,2,42.735380,0.0,NotAv,5,2007,7.344784e+05
2,RL,1045.15875,IR1,7,5,2001,Ex,Y,85.470760,80.453998,...,2,6,1,2,56.485024,0.0,NotAv,9,2008,9.074100e+05
3,RL,887.22365,IR1,7,5,1915,Gd,Y,89.279783,70.234668,...,1,7,1,3,59.643726,0.0,NotAv,2,2006,5.684000e+05
4,RL,1324.79678,IR1,8,5,2000,Ex,Y,106.373935,97.826859,...,2,9,1,3,77.666908,0.0,NotAv,12,2008,1.015000e+06


In [3]:
df_train.columns

Index(['zoneamento', 'tam_terreno', 'forma_terreno', 'qualidade_geral',
       'condicao', 'ano_construcao', 'qualidade_aquecedor', 'ar_condicionado',
       'tam_primeiro_andar', 'tam_segundo_andar', 'tam_sala_estar',
       'qtde_quartos', 'qualidade_cozinha', 'qtde_banheiros', 'qtde_comodos',
       'qtde_lareiras', 'qtde_carros_garagem', 'tam_garagem', 'tam_piscina',
       'qualidade_piscina', 'mes_venda', 'ano_venda', 'preco'],
      dtype='object')

In [4]:
# Changing to en-us column names
df_train.rename(columns={
    'zoneamento': 'zoning',
    'tam_terreno': 'land_size',
    'forma_terreno': 'land_shape',
    'qualidade_geral': 'overall_quality',
    'condicao': 'condition',
    'ano_construcao': 'year_built',
    'qualidade_aquecedor': 'heater_quality',
    'ar_condicionado': 'air_conditioning',
    'tam_primeiro_andar': 'first_floor_size',
    'tam_segundo_andar': 'second_floor_size',
    'tam_sala_estar': 'living_room_size',
    'qtde_quartos': 'number_of_bedrooms',
    'qualidade_cozinha': 'kitchen_quality',
    'qtde_banheiros': 'number_of_bathrooms',
    'qtde_comodos': 'number_of_rooms',
    'qtde_lareiras': 'number_of_fireplaces',
    'qtde_carros_garagem': 'garage_capacity',
    'tam_garagem': 'garage_size',
    'tam_piscina': 'pool_size',
    'qualidade_piscina': 'pool_quality',
    'mes_venda': 'sale_month',
    'ano_venda': 'sale_year',
    'preco': 'price'
}, inplace=True)

# Display the renamed DataFrame
print(df_train.head())

  zoning   land_size land_shape  overall_quality  condition  year_built  \
0     RL   785.03035        Reg                7          5        2003   
1     RL   891.86880        Reg                6          8        1976   
2     RL  1045.15875        IR1                7          5        2001   
3     RL   887.22365        IR1                7          5        1915   
4     RL  1324.79678        IR1                8          5        2000   

  heater_quality air_conditioning  first_floor_size  second_floor_size  ...  \
0             Ex                Y         79.524968          79.339162  ...   
1             Ex                Y        117.243586           0.000000  ...   
2             Ex                Y         85.470760          80.453998  ...   
3             Gd                Y         89.279783          70.234668  ...   
4             Ex                Y        106.373935          97.826859  ...   

   number_of_bathrooms  number_of_rooms number_of_fireplaces  garage_capac

In [5]:
target = 'price'
cat_vars = df_train.select_dtypes(include='object').columns.to_list()
num_vars = [col for col in df_train.columns if col not in cat_vars + ['Id', target]]

X = df_train.filter(cat_vars + num_vars).copy()
y = df_train[target].copy()


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=30)

In [6]:
!pip install feature-engine catboost lightgbm xgboost

In [7]:
y_train

482      537950.0
403      609000.0
161      418992.0
109      730800.0
814      556220.0
          ...    
919      592354.0
1164     996121.0
500      568400.0
941      580580.0
421     1278900.0
Name: price, Length: 1166, dtype: float64

In [8]:
random_state = 42

linear_models = [
        ('lr', LinearRegression()),
        ('ridge', Ridge(random_state=random_state, )),
        ('sgd', SGDRegressor(random_state=random_state)),
        ('svr', SVR())
]

# linear models do not change (both classification and regression)
steps_linear_models = [
    ('numeric_imputer', MeanMedianImputer(variables=num_vars, imputation_method='mean')), 
    ('numeric_scaler', SklearnTransformerWrapper(variables=num_vars, transformer=StandardScaler())), 
    ('categoric_imputer', CategoricalImputer(variables=cat_vars, fill_value='missing')), 
    ('one_hot_encoder', OneHotEncoder(variables=cat_vars))
]

# we change the tree models (Classifier -> Regressor) to regression
tree_models = [
        ('decision_tree', DecisionTreeRegressor(random_state=random_state)),
        ('random_forest', RandomForestRegressor(random_state=random_state)),
        ('gb', GradientBoostingRegressor(random_state=random_state)),
        ('xgb', XGBRegressor(random_state=random_state)),
        ('lgbm', LGBMRegressor(random_state=random_state)),
        ('catboost', CatBoostRegressor(random_state=random_state))
]

steps_tree_models = [
    ('numeric_imputer', MeanMedianImputer(variables=num_vars, imputation_method='mean')), 
    ('categoric_imputer', CategoricalImputer(variables=cat_vars, fill_value='missing')), 
    ('one_hot_encoder', OneHotEncoder(variables=cat_vars))
]

In [9]:
kfold = KFold(n_splits=5, shuffle=True, random_state=random_state)

In [10]:
def model_train(modelo, steps, X_train, y_train, cv, metrics):
    # Wrap preprocessing steps and model into a pipeline
    pipeline = Pipeline(steps + [modelo])  # Combine preprocessing steps with the model
    
    # Perform cross-validation for each metric
    results = {}
    for metric in metrics:
        scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring=metric)
        results[metric] = np.mean(scores)
    
    # Return metrics in the order of ['r2', 'mae', 'rmse']
    return [results['r2'], -results['neg_mean_absolute_error'], -results['neg_root_mean_squared_error']]

In [11]:
# List all available scorers
aux = get_scorer_names()

In [12]:
# Print all scoring names
print("Available scoring functions:")
for scorer in aux:
    print(scorer)

Available scoring functions:
accuracy
adjusted_mutual_info_score
adjusted_rand_score
average_precision
balanced_accuracy
completeness_score
d2_absolute_error_score
explained_variance
f1
f1_macro
f1_micro
f1_samples
f1_weighted
fowlkes_mallows_score
homogeneity_score
jaccard
jaccard_macro
jaccard_micro
jaccard_samples
jaccard_weighted
matthews_corrcoef
mutual_info_score
neg_brier_score
neg_log_loss
neg_max_error
neg_mean_absolute_error
neg_mean_absolute_percentage_error
neg_mean_gamma_deviance
neg_mean_poisson_deviance
neg_mean_squared_error
neg_mean_squared_log_error
neg_median_absolute_error
neg_negative_likelihood_ratio
neg_root_mean_squared_error
neg_root_mean_squared_log_error
normalized_mutual_info_score
positive_likelihood_ratio
precision
precision_macro
precision_micro
precision_samples
precision_weighted
r2
rand_score
recall
recall_macro
recall_micro
recall_samples
recall_weighted
roc_auc
roc_auc_ovo
roc_auc_ovo_weighted
roc_auc_ovr
roc_auc_ovr_weighted
top_k_accuracy
v_measure

In [13]:
metrics_regression = ['r2', 'neg_median_absolute_error', 'neg_root_mean_squared_error']

In [14]:
# Define the metrics for evaluation
metrics_regression = ['r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error']

# Define cross-validation strategy
kfold = KFold(n_splits=5, shuffle=True, random_state=random_state)

# Initialize results DataFrame
df_results = pd.DataFrame(columns=['r2', 'mae', 'rmse'])  # Ensure column names match returned values

# Training linear models
for model in linear_models:
    model_name = model[1].__class__.__name__  # Use the model class name as the identifier
    print(f'Training {model_name} ...', end=' ')
    
    # Train the model and retrieve metrics
    result = model_train(
        modelo=model, 
        steps=steps_linear_models, 
        X_train=X_train, 
        y_train=y_train, 
        cv=kfold, 
        metrics=metrics_regression
    )
    
    # Ensure the result matches the columns in df_results
    df_results.loc[model_name] = result  # result should match ['r2', 'mae', 'rmse']
    print('OK')

# Training tree models
for model in tree_models:
    model_name = model[1].__class__.__name__  # Use the model class name as the identifier
    print(f'Training {model_name} ...', end=' ')
    
    # Train the model and retrieve metrics
    result = model_train(
        modelo=model, 
        steps=steps_tree_models, 
        X_train=X_train, 
        y_train=y_train, 
        cv=kfold, 
        metrics=metrics_regression
    )
    
    # Ensure the result matches the columns in df_results
    df_results.loc[model_name] = result  # result should match ['r2', 'mae', 'rmse']
    print('OK')

# View the results DataFrame
print(df_results)

Training LinearRegression ... OK
Training Ridge ... OK
Training SGDRegressor ... OK
Training SVR ... OK
Training DecisionTreeRegressor ... OK
Training RandomForestRegressor ... OK
Training GradientBoostingRegressor ... OK
Training XGBRegressor ... 

/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 140, in __call__
    score = scorer._score(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
    result, _ = _get_response_values(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/_response.py", line 242, in _get_response_values
    y_pred, pos_label = prediction_method(X), None
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 788, in predic

/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 140, in __call__
    score = scorer._score(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
    result, _ = _get_response_values(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/_response.py", line 242, in _get_response_values
    y_pred, pos_label = prediction_method(X), None
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 788, in predic

/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 140, in __call__
    score = scorer._score(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
    result, _ = _get_response_values(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/_response.py", line 242, in _get_response_values
    y_pred, pos_label = prediction_method(X), None
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 788, in predic

/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 140, in __call__
    score = scorer._score(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
    result, _ = _get_response_values(
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/_response.py", line 242, in _get_response_values
    y_pred, pos_label = prediction_method(X), None
  File "/Users/dellacorte/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 788, in predic

...

86:	learn: 107702.1831997	total: 70.7ms	remaining: 742ms
87:	learn: 107173.1680314	total: 71.8ms	remaining: 745ms
88:	learn: 106677.7429714	total: 72.7ms	remaining: 744ms
89:	learn: 106310.5147712	total: 73.5ms	remaining: 743ms
90:	learn: 105843.5433670	total: 74.2ms	remaining: 742ms
91:	learn: 105732.2000858	total: 74.7ms	remaining: 737ms
92:	learn: 105249.9277675	total: 75.4ms	remaining: 736ms
93:	learn: 105018.7786526	total: 76.2ms	remaining: 735ms
94:	learn: 104912.2925656	total: 77ms	remaining: 733ms
95:	learn: 104435.4499398	total: 77.8ms	remaining: 732ms
96:	learn: 104015.3346231	total: 78.5ms	remaining: 731ms
97:	learn: 103369.4787855	total: 79.3ms	remaining: 730ms
98:	learn: 102922.6983318	total: 80.1ms	remaining: 729ms
99:	learn: 102764.0158335	total: 81ms	remaining: 729ms
100:	learn: 102341.7575983	total: 82ms	remaining: 730ms
101:	learn: 101868.1873008	total: 82.8ms	remaining: 729ms
102:	learn: 101653.8812353	total: 83.7ms	remaining: 729ms
103:	learn: 101208.2531396	total: 

308:	learn: 67570.4240818	total: 252ms	remaining: 564ms
309:	learn: 67538.9951736	total: 253ms	remaining: 564ms
310:	learn: 67441.7880520	total: 254ms	remaining: 563ms
311:	learn: 67308.7829212	total: 255ms	remaining: 562ms
312:	learn: 67185.2326424	total: 256ms	remaining: 562ms
313:	learn: 67123.9001073	total: 257ms	remaining: 561ms
314:	learn: 67040.6648891	total: 257ms	remaining: 560ms
315:	learn: 66922.6540144	total: 258ms	remaining: 559ms
316:	learn: 66815.0117261	total: 259ms	remaining: 558ms
317:	learn: 66786.3752702	total: 260ms	remaining: 557ms
318:	learn: 66645.5097144	total: 261ms	remaining: 556ms
319:	learn: 66602.5771120	total: 261ms	remaining: 555ms
320:	learn: 66562.0968873	total: 262ms	remaining: 555ms
321:	learn: 66442.1123763	total: 263ms	remaining: 554ms
322:	learn: 66288.6503430	total: 264ms	remaining: 553ms
323:	learn: 66199.5571956	total: 265ms	remaining: 553ms
324:	learn: 66094.7173584	total: 266ms	remaining: 552ms
325:	learn: 66052.2167567	total: 267ms	remaining

537:	learn: 47948.4285095	total: 445ms	remaining: 382ms
538:	learn: 47856.6523246	total: 447ms	remaining: 382ms
539:	learn: 47760.1439538	total: 447ms	remaining: 381ms
540:	learn: 47695.5806943	total: 448ms	remaining: 380ms
541:	learn: 47598.7226473	total: 449ms	remaining: 379ms
542:	learn: 47591.4382117	total: 449ms	remaining: 378ms
543:	learn: 47575.5241608	total: 450ms	remaining: 377ms
544:	learn: 47529.9795757	total: 451ms	remaining: 377ms
545:	learn: 47467.2064963	total: 452ms	remaining: 376ms
546:	learn: 47403.9685393	total: 453ms	remaining: 375ms
547:	learn: 47321.8574156	total: 453ms	remaining: 374ms
548:	learn: 47231.5569079	total: 454ms	remaining: 373ms
549:	learn: 47203.1083184	total: 455ms	remaining: 372ms
550:	learn: 47106.7492254	total: 456ms	remaining: 371ms
551:	learn: 47042.4036998	total: 457ms	remaining: 371ms
552:	learn: 46981.7774760	total: 458ms	remaining: 370ms
553:	learn: 46943.5687890	total: 459ms	remaining: 369ms
554:	learn: 46880.5619205	total: 460ms	remaining

765:	learn: 37154.8908249	total: 639ms	remaining: 195ms
766:	learn: 37078.0983295	total: 640ms	remaining: 195ms
767:	learn: 37032.3334292	total: 641ms	remaining: 194ms
768:	learn: 37008.6810245	total: 642ms	remaining: 193ms
769:	learn: 36969.5908658	total: 643ms	remaining: 192ms
770:	learn: 36951.6388916	total: 644ms	remaining: 191ms
771:	learn: 36889.7600816	total: 644ms	remaining: 190ms
772:	learn: 36855.1506786	total: 645ms	remaining: 189ms
773:	learn: 36798.9495230	total: 646ms	remaining: 189ms
774:	learn: 36766.9883507	total: 647ms	remaining: 188ms
775:	learn: 36719.3491615	total: 648ms	remaining: 187ms
776:	learn: 36674.8269664	total: 649ms	remaining: 186ms
777:	learn: 36632.2393032	total: 650ms	remaining: 186ms
778:	learn: 36582.5974897	total: 651ms	remaining: 185ms
779:	learn: 36531.8666442	total: 652ms	remaining: 184ms
780:	learn: 36501.7662319	total: 653ms	remaining: 183ms
781:	learn: 36476.9475550	total: 654ms	remaining: 182ms
782:	learn: 36434.3832653	total: 655ms	remaining

982:	learn: 30374.7296461	total: 833ms	remaining: 14.4ms
983:	learn: 30366.8052685	total: 834ms	remaining: 13.6ms
984:	learn: 30336.2820062	total: 835ms	remaining: 12.7ms
985:	learn: 30316.0284786	total: 836ms	remaining: 11.9ms
986:	learn: 30300.3424872	total: 836ms	remaining: 11ms
987:	learn: 30250.4852866	total: 837ms	remaining: 10.2ms
988:	learn: 30215.9303332	total: 838ms	remaining: 9.32ms
989:	learn: 30207.5919640	total: 839ms	remaining: 8.47ms
990:	learn: 30173.3743610	total: 840ms	remaining: 7.62ms
991:	learn: 30125.5765703	total: 840ms	remaining: 6.78ms
992:	learn: 30085.9835271	total: 841ms	remaining: 5.93ms
993:	learn: 30042.5216377	total: 842ms	remaining: 5.08ms
994:	learn: 30033.1979359	total: 843ms	remaining: 4.24ms
995:	learn: 29997.4646228	total: 844ms	remaining: 3.39ms
996:	learn: 29996.4598024	total: 845ms	remaining: 2.54ms
997:	learn: 29970.2176798	total: 846ms	remaining: 1.69ms
998:	learn: 29945.0339406	total: 847ms	remaining: 847us
999:	learn: 29923.5741149	total: 8

In [15]:
pd.options.display.float_format = '{:.4f}'.format

In [16]:
df_results

,r2,mae,rmse
LinearRegression,0.7481,92229.5089,158493.9641
Ridge,0.7694,90317.1621,150117.7205
SGDRegressor,0.7802,90758.3693,147343.4770
SVR,-0.0526,227259.7851,330092.1455
DecisionTreeRegressor,0.7173,112652.8644,169970.8388
RandomForestRegressor,0.8314,80499.9674,129737.2237
GradientBoostingRegressor,0.8486,75036.8350,121989.8615
XGBRegressor,NaN,NaN,NaN
LGBMRegressor,0.8440,78860.7764,126136.4564
CatBoostRegressor,0.8696,72769.1425,115522.4619


In [17]:
df_results.sort_values(['mae', 'r2'], ascending=False)

,r2,mae,rmse
SVR,-0.0526,227259.7851,330092.1455
DecisionTreeRegressor,0.7173,112652.8644,169970.8388
LinearRegression,0.7481,92229.5089,158493.9641
SGDRegressor,0.7802,90758.3693,147343.4770
Ridge,0.7694,90317.1621,150117.7205
RandomForestRegressor,0.8314,80499.9674,129737.2237
LGBMRegressor,0.8440,78860.7764,126136.4564
GradientBoostingRegressor,0.8486,75036.8350,121989.8615
CatBoostRegressor,0.8696,72769.1425,115522.4619
XGBRegressor,NaN,NaN,NaN


In [25]:
# Explicitly build the pipeline with unique names
best_model = Pipeline(steps=steps_tree_models + [('regressor', CatBoostRegressor(random_state=random_state))])

# Fit the pipeline
best_model.fit(X_train, y_train)

Learning rate set to 0.041949
0:	learn: 313206.2685733	total: 8.36ms	remaining: 8.35s
1:	learn: 304612.4880185	total: 9.32ms	remaining: 4.65s
2:	learn: 296516.2763744	total: 10.1ms	remaining: 3.35s
3:	learn: 288716.2919648	total: 11ms	remaining: 2.74s
4:	learn: 280997.4791984	total: 12.7ms	remaining: 2.52s
5:	learn: 273185.4996949	total: 13.7ms	remaining: 2.27s
6:	learn: 266705.4568795	total: 14.4ms	remaining: 2.05s
7:	learn: 260182.1344448	total: 15.2ms	remaining: 1.88s
8:	learn: 253783.8154640	total: 16ms	remaining: 1.76s
9:	learn: 247846.3402680	total: 16.9ms	remaining: 1.67s
10:	learn: 241956.7782467	total: 18ms	remaining: 1.62s
11:	learn: 236141.0254668	total: 18.8ms	remaining: 1.54s
12:	learn: 231037.0401253	total: 19.7ms	remaining: 1.49s
13:	learn: 226473.5608219	total: 21ms	remaining: 1.48s
14:	learn: 221511.5485306	total: 21.9ms	remaining: 1.44s
15:	learn: 216259.7837791	total: 22.6ms	remaining: 1.39s
16:	learn: 211844.6179040	total: 23.3ms	remaining: 1.35s
17:	learn: 207260.3

210:	learn: 81029.0928428	total: 182ms	remaining: 681ms
211:	learn: 80967.1489859	total: 183ms	remaining: 681ms
212:	learn: 80938.0611071	total: 184ms	remaining: 680ms
213:	learn: 80805.0337859	total: 185ms	remaining: 679ms
214:	learn: 80754.1562632	total: 186ms	remaining: 678ms
215:	learn: 80664.4234382	total: 187ms	remaining: 677ms
216:	learn: 80551.4328505	total: 187ms	remaining: 676ms
217:	learn: 80480.4939129	total: 188ms	remaining: 674ms
218:	learn: 80412.7241120	total: 189ms	remaining: 673ms
219:	learn: 80327.9357014	total: 190ms	remaining: 672ms
220:	learn: 80199.9321752	total: 190ms	remaining: 670ms
221:	learn: 80113.0463750	total: 191ms	remaining: 670ms
222:	learn: 80064.2885201	total: 192ms	remaining: 669ms
223:	learn: 79944.3459409	total: 193ms	remaining: 667ms
224:	learn: 79810.6917880	total: 193ms	remaining: 666ms
225:	learn: 79616.4535913	total: 194ms	remaining: 664ms
226:	learn: 79500.9545644	total: 195ms	remaining: 663ms
227:	learn: 79410.2096949	total: 195ms	remaining

452:	learn: 58647.8120986	total: 377ms	remaining: 455ms
453:	learn: 58560.9333008	total: 378ms	remaining: 455ms
454:	learn: 58498.2917895	total: 379ms	remaining: 454ms
455:	learn: 58392.0016519	total: 380ms	remaining: 453ms
456:	learn: 58327.5369258	total: 381ms	remaining: 452ms
457:	learn: 58293.7237908	total: 382ms	remaining: 451ms
458:	learn: 58225.0640330	total: 382ms	remaining: 450ms
459:	learn: 58119.4664303	total: 383ms	remaining: 450ms
460:	learn: 58052.2497034	total: 384ms	remaining: 449ms
461:	learn: 57949.5453972	total: 384ms	remaining: 448ms
462:	learn: 57907.7331440	total: 385ms	remaining: 447ms
463:	learn: 57797.0399443	total: 386ms	remaining: 446ms
464:	learn: 57788.0870818	total: 387ms	remaining: 445ms
465:	learn: 57738.1382818	total: 388ms	remaining: 444ms
466:	learn: 57694.8378992	total: 388ms	remaining: 443ms
467:	learn: 57632.0897497	total: 389ms	remaining: 442ms
468:	learn: 57608.5735910	total: 390ms	remaining: 442ms
469:	learn: 57493.8346494	total: 391ms	remaining

687:	learn: 44913.7694075	total: 572ms	remaining: 259ms
688:	learn: 44851.8596691	total: 574ms	remaining: 259ms
689:	learn: 44802.0467346	total: 576ms	remaining: 259ms
690:	learn: 44721.6642212	total: 577ms	remaining: 258ms
691:	learn: 44674.7092751	total: 578ms	remaining: 257ms
692:	learn: 44646.5619116	total: 579ms	remaining: 256ms
693:	learn: 44585.8102651	total: 580ms	remaining: 256ms
694:	learn: 44531.8483219	total: 581ms	remaining: 255ms
695:	learn: 44477.4374708	total: 581ms	remaining: 254ms
696:	learn: 44441.9375896	total: 583ms	remaining: 253ms
697:	learn: 44377.8325499	total: 584ms	remaining: 252ms
698:	learn: 44333.6903634	total: 584ms	remaining: 252ms
699:	learn: 44285.5120870	total: 585ms	remaining: 251ms
700:	learn: 44240.6838862	total: 586ms	remaining: 250ms
701:	learn: 44166.2294295	total: 587ms	remaining: 249ms
702:	learn: 44138.5229264	total: 587ms	remaining: 248ms
703:	learn: 44102.6545544	total: 588ms	remaining: 247ms
704:	learn: 44080.2736353	total: 589ms	remaining

927:	learn: 35416.9225727	total: 763ms	remaining: 59.2ms
928:	learn: 35406.7080518	total: 764ms	remaining: 58.4ms
929:	learn: 35367.6354445	total: 765ms	remaining: 57.6ms
930:	learn: 35348.0290466	total: 766ms	remaining: 56.8ms
931:	learn: 35299.5234479	total: 767ms	remaining: 56ms
932:	learn: 35255.6542323	total: 768ms	remaining: 55.1ms
933:	learn: 35206.8490363	total: 768ms	remaining: 54.3ms
934:	learn: 35194.3652319	total: 769ms	remaining: 53.5ms
935:	learn: 35175.8017751	total: 770ms	remaining: 52.7ms
936:	learn: 35145.8061571	total: 771ms	remaining: 51.8ms
937:	learn: 35130.6389800	total: 772ms	remaining: 51ms
938:	learn: 35091.5558206	total: 773ms	remaining: 50.2ms
939:	learn: 35062.8407509	total: 774ms	remaining: 49.4ms
940:	learn: 35023.2786148	total: 775ms	remaining: 48.6ms
941:	learn: 35014.1475327	total: 776ms	remaining: 47.8ms
942:	learn: 35000.7805579	total: 777ms	remaining: 47ms
943:	learn: 34963.0929404	total: 778ms	remaining: 46.1ms
944:	learn: 34924.4032281	total: 778m

Pipeline(steps=[('numeric_imputer',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['land_size', 'overall_quality',
                                              'condition', 'year_built',
                                              'first_floor_size',
                                              'second_floor_size',
                                              'living_room_size',
                                              'number_of_bedrooms',
                                              'number_of_bathrooms',
                                              'number_of_rooms',
                                              'number_of_fireplaces',
                                              'garage_capacity', 'garage_size',
                                              'pool_size', 'sale_month',
                                              'sale_y...
                 CategoricalImputer(fill_value='missing',
                                    variables=['zoning', 'land_shape',
                                               'heater_quality',
                                               'air_conditioning',
                                               'kitchen_quality',
                                               'pool_quality'])),
                ('one_hot_encoder',
                 OneHotEncoder(variables=['zoning', 'land_shape',
                                          'heater_quality', 'air_conditioning',
                                          'kitchen_quality', 'pool_quality'])),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x7fb8101781f0>)])

In [26]:
print(best_model)

Pipeline(steps=[('numeric_imputer',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['land_size', 'overall_quality',
                                              'condition', 'year_built',
                                              'first_floor_size',
                                              'second_floor_size',
                                              'living_room_size',
                                              'number_of_bedrooms',
                                              'number_of_bathrooms',
                                              'number_of_rooms',
                                              'number_of_fireplaces',
                                              'garage_capacity', 'garage_size',
                                              'pool_size', 'sale_month',
                                              'sale_y...
                 CategoricalImputer(fill_value='missing',
             

In [27]:
y_pred_train = best_model.predict(X_train)

In [28]:
y_pred_train[:3]

array([534230.82257058, 590553.59937245, 396550.29547198])

In [32]:
print(f'R2   Train: {r2_score(y_train, y_pred_train)}')
print(f'MAE  Train: {mean_absolute_error(y_train, y_pred_train)}')
print(f'RMSE Train: {mean_squared_error(y_train, y_pred_train)}')

R2   Train: 0.9893193585497878
MAE  Train: 25789.76982800545
RMSE Train: 1110823423.2982476


In [34]:
y_pred_test = best_model.predict(X_test)

In [36]:
print(f'R2   Teste: {r2_score(y_test, y_pred_test)}')
print(f'MAE  Teste: {mean_absolute_error(y_test, y_pred_test)}')
print(f'RMSE Teste: {mean_squared_error(y_test, y_pred_test)}')

R2   Teste: 0.8175783859604329
MAE  Teste: 68000.82595696872
RMSE Teste: 18818160990.239384


In [38]:
print(f'MAPE Test: {mean_absolute_percentage_error(y_test, y_pred_test)}')

MAPE Test: 0.10128299225724308


In [42]:
def adjusted_r2(y_expected, y_predict, X_train):
    R2 = r2_score(y_expected, y_predict)
    N = len(y_expected)
    p = X_train.shape[1]

    adjusted_r2 = (1 - ((1 - R2) * (N - 1)) / (N - p - 1))
    return adjusted_r2

adjusted_r2 = adjusted_r2(y_test, y_pred_test, X_train)
print('R2 Ajustado:', adjusted_r2)

mae = mean_absolute_error(y_test, y_pred_test)
print('MAE:', mae)

mse = mean_squared_error(y_test, y_pred_test)
print('MSE:', mse)

rmse = mean_squared_error(y_test, y_pred_test)
print('RMSE:', rmse)

rmsle = mean_squared_log_error(y_test, y_pred_test)
print('RMSLE:', rmsle)

m_error = max_error(y_test, y_pred_test)
print('Max Error:', m_error)

mape = mean_absolute_percentage_error(y_test, y_pred_test)
print('MAPE:', mape)

R2 Ajustado: 0.8026591461505055
MAE: 68000.82595696872
MSE: 18818160990.239384
RMSE: 18818160990.239384
RMSLE: 0.021596631122553462
Max Error: 1769442.423498673
MAPE: 0.10128299225724308


In [ ]:
#